In [5]:
import pandas as pd
import numpy as np

import lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from collections import Counter
import warnings
warnings.filterwarnings('ignore')
import pre_process
from sklearn.model_selection  import train_test_split

In [2]:
#读取相关文件
train_path = '../data/train.csv'
test_path = '../data/test.csv'
submit_path = '../data/车辆贷款违约预测挑战赛sample_submit.csv'
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
submit_data = pd.read_csv(submit_path)
#生成提交文件
submit_data['customer_id'] = test_data['customer_id']
submit_data['loan_default'] = 0

In [3]:
train_data,test_data = pre_process.fill_inf(train_data,test_data)#填补inf值
train_data,test_data = pre_process.del_singular_feature(train_data,test_data)#删除单值属性

In [6]:
#划分数据集
labels = train_data['loan_default'].values
features_scaler = train_data.drop(['loan_default'],axis=1).values
X_train,X_test, y_train, y_test = train_test_split(features_scaler, labels, test_size = 0.2, random_state = 0)

In [9]:
from lightgbm import LGBMClassifier
#第一步：学习率和迭代次数
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
# 切分数据 训练数据80% 验证数据20%
# 为了加快速度CV选的3，其实一般用5
# 因为每训练一次耗时很多，所以每个参数的选项不多，间隔比较大，正式的时候应该是比较多，间隔比较细的
# 本次只是演示，所以如果最好参数位于区间的边缘也就直接用了，其实如果最好参数在边缘，需要重新再搜索。
model = LGBMClassifier(boosting_type='gbdt',
                       objective='binary',
                       metrics='auc',
                       learning_rate=0.1,
                       max_depth=5,
                       bagging_fraction=0.8,
                       feature_fraction=0.8)
parameters = {'n_estimators': [100, 150, 175, 200, 225, 250]}

clf = GridSearchCV(model, parameters, cv=3, verbose=2)
clf.fit(X_train, y_train)
score_test = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])

print("LightGBM GridSearchCV AUC Score:   ", score_test)
print("最优参数:")
print(clf.best_params_)


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[CV] END ...................................n_estimators=100; total time=   1.3s
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[CV] END ...................................n_estimators=100; total time=   1.2s
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[CV] END ...................................n_estimator